In [ ]:
%pip install eyepop==3.9.10

## Setting authentication variables

In [ ]:
import getpass

EYEPOP_ACCOUNT_ID="7587fa529fad497c8965d7b6a81a0010"
EYEPOP_API_KEY="eyp_9b33b6bf30501dce141ef92c02e4c14ab608facad9e0dc46bd43b7f18d2e2872a39c44c72bb9bac87e31ccec1cc057128b5e872fa0db793bce7b86dd9a3b4dc3"

## Setting a namespace prefix
All aliases for abilities will be created with this prefix. NOTE: the account must have a namespace prefix registered so that this values starts with the registered prefix. E.g. if you account has a namespace prefix registered like `foo.bar` you can use here `foo.bar` or `foo.bar.baz`. 

In [ ]:
NAMESPACE_PREFIX="carsandbids"

## Defining the abilities
This just defines the list of abilities as prototypes. The registration happens below.

In [ ]:
from eyepop import EyePopSdk
from eyepop.data.data_types import InferRuntimeConfig, VlmAbilityGroupCreate, VlmAbilityCreate, TransformInto
from eyepop.worker.worker_types import CropForward, ForwardComponent, FullForward, InferenceComponent, Pop
import json

ability_prototypes = [
    VlmAbilityCreate(
        name=f"{NAMESPACE_PREFIX}.image-classify.primary-shot-type",
        description="",
        worker_release="qwen3-instruct",
        text_prompt="Determine the primary content of this image and assign exactly one label: Exterior, Interior, Document, or Other. Choose Exterior only if the full or mostly full exterior of the vehicle is visible in a standard photo. Choose Interior if the vehicle's cabin is visible. Choose Document if the image contains documents or forms with no vehicle visible. Choose Other if the image shows a close-up detail of the vehicle, a partial component, the undercarriage, or anything that does not meet the definitions above. Return only the label.",
        transform_into=TransformInto(
            classes=[
                "Exterior", 
                "Interior", 
                "Document", 
                "Other"
            ]
        ),
        config=InferRuntimeConfig(
            max_new_tokens=5,
            image_size=512
        ),
        is_public=False
    ),
    VlmAbilityCreate(
        name=f"{NAMESPACE_PREFIX}.image-classify.subtype.exterior",
        description="",
        worker_release="qwen3-instruct",
        text_prompt="Determine the exterior shot type for this vehicle image. The image is guaranteed to be an Exterior view. Select exactly one label from the following list and return only that label: front profile, front ¾ view, side profile, rear ¾ view, rear profile, convertible top up, convertible top down. Do not infer mirrored or reflected views; do not infer hidden parts or multiple views; do not return more than one label; do not include explanations or additional text. Return only the label exactly as written above.",
        transform_into=TransformInto(
            classes=[
                "front profile",
                "front ¾ view",
                "side profile",
                "rear ¾ view",
                "rear profile",
                "convertible top up",
                "convertible top down"
            ]
        ),
        config=InferRuntimeConfig(
            max_new_tokens=5,
            image_size=512
        ),
        is_public=False
    ),
    VlmAbilityCreate(
        name=f"{NAMESPACE_PREFIX}.image-classify.subtype.exterior-left-right",
        description="",
        worker_release="qwen3-instruct",
        text_prompt="Is the front of this vehicle pointed to the left or right of the image? Respond with LEFT or RIGHT only.",
        transform_into=TransformInto(
            classes=[
                "LEFT",
                "RIGHT",
            ]
        ),
        config=InferRuntimeConfig(
            max_new_tokens=5,
            image_size=512
        ),
        is_public=False
    ),
    VlmAbilityCreate(
        name=f"{NAMESPACE_PREFIX}.image-classify.subtype.document-ocr-title",
        description="",
        worker_release="qwen3-instruct",
        text_prompt="Transcribe the text in this image. Find the following fields and output in JSON format:\ndocument_type\nstate_two_letter\ncertificate_number\nvehicle_identification_number\nvehicle_type\nowner_name\nowner_address\ndate_of_issue\ndate_of_transfer\ntitle_number\ntitle_type\nlien_info\ntitle_brand\nrespond only with json",
        transform_into=TransformInto(),
        config=InferRuntimeConfig(
            max_new_tokens=200,
            image_size=1024
        ),
        is_public=False
    )    
]

## Listing current abilities in this account

In [ ]:
with EyePopSdk.dataEndpoint(api_key=EYEPOP_API_KEY, account_id=EYEPOP_ACCOUNT_ID, eyepop_url="https://compute.eyepop.ai") as endpoint:
    vlm_ability_groups = endpoint.list_vlm_ability_groups()
    print(f'found {len(vlm_ability_groups)} active ability groups in account {EYEPOP_ACCOUNT_ID}')
    for vlm_ability_group in vlm_ability_groups:
        vlm_abilities = endpoint.list_vlm_abilities(vlm_ability_group_uuid=vlm_ability_group.uuid)
        print(f'\nfound {len(vlm_abilities)} active abilities in group {vlm_ability_group.name} ({vlm_ability_group.uuid}):')
        for vlm_ability in vlm_abilities:
            aliases = '.'.join([f'{entry.alias}:{entry.tag}' for entry in vlm_ability.alias_entries])
            print(f'\t{vlm_ability.name} ({vlm_ability.uuid}): status={vlm_ability.status}, aliases=[{aliases}]')

## DELETE all abilities in this account

In [ ]:
with EyePopSdk.dataEndpoint(api_key=EYEPOP_API_KEY, account_id=EYEPOP_ACCOUNT_ID, eyepop_url="https://compute.eyepop.ai") as endpoint:
    vlm_ability_groups = endpoint.list_vlm_ability_groups()
    print(f'found {len(vlm_ability_groups)} active ability groups in account {EYEPOP_ACCOUNT_ID}')
    for vlm_ability_group in vlm_ability_groups:
        endpoint.delete_vlm_ability_group(vlm_ability_group.uuid)
        print(f'\tDELETED {vlm_ability_group.name} / {vlm_ability_group.uuid}')
    vlm_abilities = endpoint.list_vlm_abilities()
    print(f'found {len(vlm_abilities)} active abilities in account {EYEPOP_ACCOUNT_ID}:')
    for vlm_ability in vlm_abilities:
        endpoint.delete_vlm_ability(vlm_ability.uuid)
        print(f'\tDELETED {vlm_ability.name} / {vlm_ability.uuid}')

## Create all abilities, groups and aliases 

In [ ]:
with EyePopSdk.dataEndpoint(api_key=EYEPOP_API_KEY, account_id=EYEPOP_ACCOUNT_ID, eyepop_url="https://compute.eyepop.ai") as endpoint:
    for ability_prototype in ability_prototypes:
        ability_group = endpoint.create_vlm_ability_group(VlmAbilityGroupCreate(
            name=ability_prototype.name,
            description=ability_prototype.description,
            default_alias_name=ability_prototype.name,
        ))
        ability = endpoint.create_vlm_ability(
            create=ability_prototype,
            vlm_ability_group_uuid=ability_group.uuid,
        )
        ability = endpoint.publish_vlm_ability(
            vlm_ability_uuid=ability.uuid,
            alias_name=ability_prototype.name,
        )
        ability = endpoint.add_vlm_ability_alias(
            vlm_ability_uuid=ability.uuid,
            alias_name=ability_prototype.name,
            tag_name="latest"
        )
        print(f"created ability {ability.uuid} with alias entries {ability.alias_entries}")

## Define some example image Urls
<p>
    <img src="https://media.istockphoto.com/id/2187504402/photo/jaguar-f-type-svr-sports-car.jpg?s=612x612&w=0&k=20&c=NdGWdV6tWIwl95k2fw-LxVcn0ZBR73h08-7aYsrChfQ=" alt="drawing" width="200"/>
    <img src="https://preview.redd.it/current-copy-paste-of-modern-car-interior-multiple-pictures-v0-mg1buk1pcsh81.jpg?width=640&crop=smart&auto=webp&s=ac0ed732dfd82b79e16317fbf6c092d2d69cc780" alt="drawing" width="200"/>
    <img src="https://www.factorywarrantylist.com/uploads/3/3/9/4/3394652/editor/8539916.jpg?1575678451" alt="drawing" width="200"/>
</p>


In [ ]:
example_urls = [
    "https://media.istockphoto.com/id/2187504402/photo/jaguar-f-type-svr-sports-car.jpg?s=612x612&w=0&k=20&c=NdGWdV6tWIwl95k2fw-LxVcn0ZBR73h08-7aYsrChfQ=",
    "https://preview.redd.it/current-copy-paste-of-modern-car-interior-multiple-pictures-v0-mg1buk1pcsh81.jpg?width=640&crop=smart&auto=webp&s=ac0ed732dfd82b79e16317fbf6c092d2d69cc780",
    "https://www.factorywarrantylist.com/uploads/3/3/9/4/3394652/editor/8539916.jpg?1575678451"
]

## Run individual inference via Pop

In [ ]:
pop = Pop(components=[
    InferenceComponent(
        ability=f"{NAMESPACE_PREFIX}.image-classify.primary-shot-type:latest"
    )
])    
with EyePopSdk.workerEndpoint(api_key=EYEPOP_API_KEY, eyepop_url="https://compute.eyepop.ai") as endpoint:
    endpoint.set_pop(pop)
    for url in example_urls:
        job = endpoint.load_from(url)
        while result := job.predict():
            print(json.dumps(result, indent=2))
    print("done")

## Run the full composable Pop

In [ ]:
pop = Pop(components=[
    InferenceComponent(
        ability=f"{NAMESPACE_PREFIX}.image-classify.primary-shot-type:latest",
        forward=FullForward(
            targets=[ForwardComponent(
                forward=FullForward(
                    includeClasses=["Exterior"],
                    targets=[InferenceComponent(
                        ability=f"{NAMESPACE_PREFIX}.image-classify.subtype.exterior:latest",
                        categoryName="subtype_exterior"
                    ),
                    InferenceComponent(
                        ability=f"{NAMESPACE_PREFIX}.image-classify.subtype.exterior-left-right:latest",
                        categoryName="subtype_exterior_orientation"
                    )]
                )              
            ), ForwardComponent(
                forward=FullForward(
                    includeClasses=["Document"],
                    targets=[InferenceComponent(
                        ability=f"{NAMESPACE_PREFIX}.subtype.document-ocr-title:latest",
                        categoryName="document_ocr"
                    )]
                )              
            )]
        )
    )
])    
with EyePopSdk.workerEndpoint(api_key=EYEPOP_API_KEY) as endpoint:
    endpoint.set_pop(pop)
    for url in example_urls:
        job = endpoint.load_from(url)
        while result := job.predict():
            print(json.dumps(result, indent=2))